In [ ]:
import lightgbm as lgb
import numpy as np
import pandas as pd


In [ ]:
# leer en df_full ./data/l_vm_completa_train.parquet
df_full = pd.read_parquet('./data/l_vm_completa_train.parquet')
print(df_full.shape)

In [ ]:
# Variables categóricas
categorical_features = ['ANIO','MES','TRIMESTRE','ID_CAT1','ID_CAT2','ID_CAT3','ID_BRAND','SKU_SIZE','CUSTOMER_ID','PRODUCT_ID','PLAN_PRECIOS_CUIDADOS']
# Convertir las variables categóricas a tipo 'category'
for col in categorical_features:
    df_full[col] = df_full[col].astype('category')

In [ ]:

# filtrar que en X el periodo sea menor o igual a 201910
# Variables predictoras y objetivo
# En x eliminar la columna 'CLASE' y 'CLASE_DELTA'
X = df_full[df_full['PERIODO'] <= 201910].drop(columns=['CLASE', 'CLASE_DELTA']) 

In [ ]:
# Filtrar en y que el periodo sea menor o igual a 201910
y = df_full[df_full['PERIODO'] <= 201910]['CLASE']

In [ ]:
# Definir los periodos de validación 201909, 201910
periodos_valid = [201909,201910]


# Separar train y cinco conjuntos de validación respetando la secuencia temporal
X_train = X[X['PERIODO'] < periodos_valid[0]]
y_train = y[X['PERIODO'] < periodos_valid[0]]

X_val_list = [X[X['PERIODO'] == p] for p in periodos_valid]
y_val_list = [y[X['PERIODO'] == p] for p in periodos_valid]


In [ ]:
import matplotlib.pyplot as plt
plt.hist(df_full['CLASE'], bins=50)
plt.title('Distribución del target (CLASE)')
plt.show()
print(df_full['CLASE'].describe())
# Imprimir la cantidad de ejemplos por clase de los primeros 30 con mas ejemplos
print(df_full['CLASE'].value_counts().head(30))
# Calcular los ceros que porcentaje son del total
total = df_full['CLASE'].count()
ceros = df_full['CLASE'].value_counts().get(0, 0)
porcentaje_ceros = (ceros / total) * 100
print(f"Porcentaje de ceros: {porcentaje_ceros:.2f}%")



In [ ]:
# --- ETAPA 1: CLASIFICACIÓN ---
# Crear variable binaria: 0 si CLASE==0, 1 si CLASE>0
df_full['CLASE_BIN'] = (df_full['CLASE'] > 0).astype(int)

# Separar train/val como antes
X_train = X[X['PERIODO'] < periodos_valid[0]]
y_train_bin = df_full.loc[X_train.index, 'CLASE_BIN']
X_val_list = [X[X['PERIODO'] == p] for p in periodos_valid]
y_val_bin_list = [df_full.loc[X_val.index, 'CLASE_BIN'] for X_val in X_val_list]

# Entrenar clasificador
clf = lgb.LGBMClassifier(
    n_estimators=1000,
    learning_rate=0.01,
    num_leaves=64,
    n_jobs=-1
)
clf.fit(
    X_train,
    y_train_bin,
    eval_set=[(X_val_list[0], y_val_bin_list[0])],
    callbacks=[lgb.early_stopping(stopping_rounds=300), lgb.log_evaluation(period=300)],
    categorical_feature=categorical_features
)


In [ ]:
# --- ETAPA 2: REGRESIÓN SOLO PARA NO-CERO ---
# Filtrar solo donde CLASE > 0
mask_train = y_train_bin == 1
X_train_reg = X_train[mask_train]
y_train_reg = y_train[mask_train]

X_val_reg_list = []
y_val_reg_list = []
for X_val, y_val_bin, y_val in zip(X_val_list, y_val_bin_list, y_val_list):
    mask_val = y_val_bin == 1
    X_val_reg_list.append(X_val[mask_val])
    y_val_reg_list.append(y_val[mask_val])

# Entrenar regresor
reg = lgb.LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.01,
    num_leaves=64,
    n_jobs=-1
)
reg.fit(
    X_train_reg,
    y_train_reg,
    eval_set=[(X_val_reg_list[0], y_val_reg_list[0])],
    callbacks=[lgb.early_stopping(stopping_rounds=300), lgb.log_evaluation(period=300)],
    categorical_feature=categorical_features
)

In [ ]:
# --- PREDICCIÓN EN VALIDACIÓN ---
# 1. Predice probabilidad de no-cero
proba_no_cero = clf.predict_proba(X_val_list[0])[:, 1]
# 2. Predice valor solo donde proba_no_cero > umbral (ejemplo: 0.5)
umbral = 0.5
pred_bin = (proba_no_cero > umbral)
pred_reg = np.zeros(len(X_val_list[0]))
if pred_bin.sum() > 0:
    pred_reg[pred_bin] = reg.predict(X_val_list[0][pred_bin])

# 3. Calcula WAPE en validación
y_val_real = y_val_list[0].values
wape = np.sum(np.abs(y_val_real - pred_reg)) / np.sum(np.abs(y_val_real))
print(f"WAPE validación (dos etapas): {wape:.4f}")

In [ ]:
# Crear los datasets de LightGBM
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical_features)
val_data_list = [lgb.Dataset(X_val_list[i], label=y_val_list[i], categorical_feature=categorical_features) for i in range(len(periodos_valid))]


params = { 
    'objective': 'tweedie',
    'tweedie_variance_power': 1.3,
    #'metric': ['mape','mae','rmse'],
    'boosting_type': 'gbdt',
    'num_leaves': 511,
    'learning_rate': 0.005,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'max_bin': 255,               # Reduce memoria y acelera el entrenamiento
    'force_col_wise': True,       # Más eficiente en CPU para datasets anchos
    'n_jobs': -1,                 # Usa todos los núcleos disponibles
    'verbose': -1 }

def wape_metric(preds, train_data):
    y_true = train_data.get_label()
    wape = np.sum(np.abs(y_true - preds)) / np.sum(np.abs(y_true))
    return 'wape', wape, False  # False: menor es mejor

# Entrenar el modelo con validación múltiple y early stopping
model = lgb.train(
    params,
    train_data,
    num_boost_round=50000,
    valid_sets=val_data_list,
    valid_names=[f'validation_{p}' for p in periodos_valid],
    feval=wape_metric,  # <-- agrega la métrica personalizada
    callbacks=[lgb.early_stopping(stopping_rounds=300), lgb.log_evaluation(period=300)]
)

print("Modelo de regresión entrenado.")

In [ ]:
# Obtener la importancia de cada variable
importancia = model.feature_importance(importance_type='gain')
nombres = X_train.columns

# Crear un DataFrame ordenado por importancia
import pandas as pd
df_importancia = pd.DataFrame({'feature': nombres, 'importance': importancia})
df_importancia = df_importancia.sort_values(by='importance', ascending=False)

# Mostrar las variables más importantes
print(df_importancia)

# Si quieres visualizarlo gráficamente:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
plt.barh(df_importancia['feature'], df_importancia['importance'])
plt.gca().invert_yaxis()
plt.title('Importancia de variables LightGBM')
plt.xlabel('Importancia')
plt.show()